[Datashader](http://datashader.org) renders data into regularly sampled arrays, a process known as [rasterization](https://en.wikipedia.org/wiki/Rasterisation), typically just before displaying it as an image.  In many cases, your data is *already* rasterized, such as data from imaging experiments, simulations on a regular grid, or other systematic processes.  Even so, often the rasters you have already are not the ones you need for a given purpose, having the wrong shape, range, or size to be suitable for overlaying with or comparing against other data or on maps.

Datashader provides fast methods for ["regridding"](https://climatedataguide.ucar.edu/climate-data-tools-and-analysis/regridding-overview)/["re-sampling"](http://gisgeography.com/raster-resampling/)/"re-rasterizing" yuor gridded datasets, generating new rasters on demand that can be used together with those it generates for any other data types to implement complex cross-datatype analyses or visualizations.

To get started, let's declare a small raster using Numpy and wrap it up as an [xarray](http://xarray.pydata.org) DataArray: 

In [ ]:
import numpy as np, datashader as ds, xarray as xr
from datashader import transfer_functions as tf

def f(x,y):
    return np.cos((x**2+y**2)**2)

def sample(fun, n=50, range_=(0.0,2.4)):
    ls  = np.linspace(*range_, n)
    x,y = np.meshgrid(ls, ls)
    z   = f(x,y)
    return xr.DataArray(z, coords={'x': ls, 'y': ls}, dims=('y', 'x'))

da = sample(f)

DataArrays happen to be the format used for datashader's own output, so you can now immediately turn this array into an image using ``tf.shade()`` just as you would for points or lines rasterized by datashader:

In [ ]:
tf.shade(da)

So, what if we want a larger version?  We can do that by upsampling with either nearest-neighbor or bilinear interpolation (the default):

In [ ]:
tf.Images(tf.shade(ds.Canvas().raster(da),                            name="linear upsampling (default)"),
          tf.shade(ds.Canvas().raster(da, upsample_method='nearest'), name="nearest-neighbor upsampling"))

As you can see, linear interpolation works well for smoothly varying values, like those in the lower left of this function, but doesn't help much for regions close to or beyond the sampling limits of the original raster.

We can choose whatever output size we like and sample the grid over any range we like, though of course there won't be any data in regions outside of the original raster grid: 

In [ ]:
tf.shade(ds.Canvas(plot_height=200, plot_width=600, x_range=(-2,5), y_range=(-0.1,0.4)).raster(da))

The previous examples all use upsampling, from a smaller to a larger number of cells per unit distance in X or Y.  Downsampling works just as for points and lines in Datashader, supporting various aggregation functions.  These aggregation functions determine the result when more than one raster grid cell falls into a given pixel's region of the plane. To illustrate downsampling, let's first render a 500x500 version of the above 50x50 array:

In [ ]:
da2 = sample(f, n=500)
tf.shade(da2)

You can see that the version we upsampled to this size from the 50x50 samples is similar to this, but this one is much smoother and more accurately represents the underlying mathematical function, because it has sufficient resolution to avoid aliasing in the high-frequency portions of this function (towards the upper right).  Now that we have this larger array, we can downsample it using a variety of aggregation functions:

In [ ]:
cvs = ds.Canvas(plot_width=150, plot_height=150)
tf.Images(tf.shade(cvs.raster(da2),                          name="mean downsampling (default)"),
          tf.shade(cvs.raster(da2, downsample_method='min'), name="min downsampling"),
          tf.shade(cvs.raster(da2, downsample_method='max'), name="max downsampling"),
          tf.shade(cvs.raster(da2, downsample_method='std'), name="std downsampling"))

Here the default downsampling function ``mean`` renders a faithful size-reduced version of the original, with all the raster grid points that overlap a given pixel being averaged to create the final pixel value.  The ``min`` and ``max`` aggregation functions take the minimum or maxiumum, respectively, of the values overlapping each pixel, and you can see here that the ``min`` version has larger light-blue regions towards the upper right (with each pixel reflecting the minimum of all grid cells it overlaps), while the ``max`` version has larger dark-blue regions towards the upper right.  The ``std`` version reports the standard deviation of the grid cells in each pixel, which is low towards the lower left where the function is smooth, and increases towards the upper right, where the function value varies significantly per pixel.

The differences between min and max are clearer if we look at a regime where the function varies so much that it can only barely be faithfully be represented in a grid of this size:

In [ ]:
da3 = sample(f, n=500, range_=(0,3))
tf.shade(da3)

In the upper right of this plot, you can see that the function is varying with such high frequency that any downsampled version will fail to represent it properly.  The aggregation functions in Datashader can help you see when that is happening:

In [ ]:
cvs = ds.Canvas(plot_width=150, plot_height=150)
tf.Images(tf.shade(cvs.raster(da3),                          name="mean downsampling (default)"),
          tf.shade(cvs.raster(da3, downsample_method='min'), name="min downsampling"),
          tf.shade(cvs.raster(da3, downsample_method='max'), name="max downsampling"),
          tf.shade(cvs.raster(da3, downsample_method='std'), name="std downsampling"))

Here you can see that the ``mean`` downsampling looks like a good approximation to the original array, locally averaging the original function values in each portion of the array.  However, if you were to zoom in and adjust for contrast, you would be able to see some of the inevitable aliasing artifacts that occur for any such representation in a too-small array.  These aliasing effects are clearly visible in the ``min`` and ``max`` aggregation, because they keep the local minimum or maxiumum rather than averaging out the artifacts.  Comparing ``mean`` and ``min`` or ``max`` (or subtracting ``min`` from ``max``) can help find regions of the array that are being poorly represented in the current view.

In practice, it is usually a good idea to view your data interactively at many different zoom levels to detect such problems; see [3 Timeseries](3_Timeseries.ipynb).  For such cases, you can define both upsampling and downsampling methods at the same time; whichever one is needed for a given zoom level and range of the data will be applied.

You can see Datashader rasters at work in the [Landsat](../topics/Landsat.ipynb) example:

<img src="https://raw.githubusercontent.com/bokeh/datashader/master/docs/images/landsat.png" width=1204 height=1150>